In [1]:
import pandas as pd
import geopandas as gpd
import requests
import os
from shapely.geometry import Point

In [2]:
url = 'http://www2.census.gov/geo/docs/maps-data/data/gazetteer/2016_Gazetteer/2016_Gaz_zcta_national.zip'
zips = pd.read_csv(url, dtype={'GEOID' : 'str'},sep='\t', usecols=[0,5,6])
zips.columns = zips.columns.str.strip()  #some column cleanup
print (len(zips))
print (zips.head())

33144
   GEOID   INTPTLAT  INTPTLONG
0  00601  18.180555 -66.749961
1  00602  18.361945 -67.175597
2  00603  18.455183 -67.119887
3  00606  18.158345 -66.932911
4  00610  18.295366 -67.125135


In [3]:
geom = zips.apply(lambda x : Point([x['INTPTLONG'],x['INTPTLAT']]), axis=1)
zips = gpd.GeoDataFrame(zips, geometry=geom)
zips.crs = {'init' :'epsg:4326'}
print (zips.head())

   GEOID   INTPTLAT  INTPTLONG                              geometry
0  00601  18.180555 -66.749961          POINT (-66.749961 18.180555)
1  00602  18.361945 -67.175597          POINT (-67.175597 18.361945)
2  00603  18.455183 -67.119887  POINT (-67.11988700000001 18.455183)
3  00606  18.158345 -66.932911          POINT (-66.932911 18.158345)
4  00610  18.295366 -67.125135          POINT (-67.125135 18.295366)


In [5]:
geojson_file = 'data/gz_2010_us_040_00_500k.json'

if not os.path.exists(geojson_file):
    url = 'http://eric.clst.org/wupl/Stuff/gz_2010_us_040_00_500k.json'
    with open(geojson_file, 'w') as f:
        f.write(requests.get(url).text)

states = gpd.read_file(geojson_file)[['NAME', 'geometry']]
print (len(states))
print (states.head())

52
            NAME                                           geometry
0          Maine  (POLYGON ((-67.619761 44.519754, -67.61541 44....
1  Massachusetts  (POLYGON ((-70.832044 41.606504, -70.823735 41...
2       Michigan  (POLYGON ((-88.684434 48.115785, -88.675628 48...
3        Montana  POLYGON ((-104.057698 44.997431, -104.250145 4...
4         Nevada  POLYGON ((-114.0506 37.000396, -114.049995 36....


In [6]:
zips_and_states = gpd.sjoin(zips, states, op='within')
print (zips_and_states[['GEOID', 'NAME', 'geometry','index_right']].head())

   GEOID         NAME                              geometry  index_right
0  00601  Puerto Rico          POINT (-66.749961 18.180555)           16
1  00602  Puerto Rico          POINT (-67.175597 18.361945)           16
2  00603  Puerto Rico  POINT (-67.11988700000001 18.455183)           16
3  00606  Puerto Rico          POINT (-66.932911 18.158345)           16
4  00610  Puerto Rico          POINT (-67.125135 18.295366)           16
